In [1]:
import os
import time
import warnings

import cv2
import numpy as np
import pandas as pd
import time

from src.generate_patches import CropImage
from src.model_test import AntiSpoofPredict
from src.utility import parse_model_name

warnings.filterwarnings("ignore")

In [2]:
def test(image, model_dir):
    model_test = AntiSpoofPredict()
    image_cropper = CropImage()

    image_bbox = model_test.get_bbox(image)
    prediction = np.zeros((1, 2))
    # sum the prediction from single model's result
    for model_name in os.listdir(model_dir):
        h_input, w_input, model_type, scale = parse_model_name(model_name)
        param = {
            "org_img": image,
            "bbox": image_bbox,
            "scale": scale,
            "out_w": w_input,
            "out_h": h_input,
            "crop": True,
        }
        if scale is None:
            param["crop"] = False
        img = image_cropper.crop(**param)
        prediction += model_test.predict(img, os.path.join(model_dir, model_name))

    label = np.argmax(prediction)
    value = prediction[0][label] / 2
    
    if label == 1:
        return value
    else:
        return 1 - value


In [3]:
def post_processing(file_path,model_dir):
    cap = cv2.VideoCapture(file_path)
    ls = []
    c = 0
    while cap.isOpened():
        ret, frame = cap.read()
        try:
            if c % 5 == 0:
                score = test(frame, model_dir)
                ls.append(score)
            c+=1
        except:
            break
    return ls

In [4]:
test_cases = os.listdir('/home/eco0936_namnh/CODE/zalo-challenge/data/')
model_dir = './resources/ckpt/'

In [5]:
all_predicted_time = []
all_result = []
print('preprocessing ...')
for file_name in test_cases:
    t1 = time.time()  
    file_path = os.path.join('/home/eco0936_namnh/CODE/zalo-challenge/data/',file_name)
    result = post_processing(file_path,model_dir)
    t2 = time.time()
    predicted_time = int(t2*1000 - t1*1000)
    all_predicted_time.append((file_name, predicted_time))
    all_result.append((file_name,sum(result)/len(result)))
df_1 = pd.DataFrame(all_result,columns=["fname", "liveness_score"])
df_2 = pd.DataFrame(all_predicted_time,columns=["fname", "time_submission"])
os.makedirs('results/',exist_ok=True)
df_1.to_csv("results/jupyter_submission.csv", index=False, encoding="utf-8", float_format="%.10f")
df_2.to_csv("results/time_submission.csv", index=False, encoding="utf-8", float_format="%.10f")

print('output will be saved in /result/time_submission.csv')
print('output will be saved in /result/jupyter_submission.csv')

preprocessing ...
output will be saved in /result/time_submission.csv
output will be saved in /result/jupyter_submission.csv
